# Reflectance Canonical Dataset — Sample Walkthrough

Use this notebook to inspect the canonical reflectance tables generated by the pipeline. The sample files in `data-sample/reflectance_canonical/` provide quick feedback without needing the full dataset.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

project_root = Path.cwd()
sys.path.append(str(project_root / 'src'))

sample_dir = project_root / 'data-sample' / 'reflectance_canonical'

precision = pd.read_csv(sample_dir / 'precision_weighted_concentrations_sample.csv')
dose_summary = pd.read_csv(sample_dir / 'dose_summary_sample.csv')
print(f"Sample rows: precision={len(precision)}, dose_summary={len(dose_summary)}")
precision.head()


Extract the trimmed reflectance spectra (per sample and angle) to compute region-of-interest occupancy.

In [ ]:
trimmed = pd.read_csv(sample_dir / 'reflectance_trimmed_stats_sample.csv')
roi_cols = [c for c in trimmed.columns if c.startswith('mean_')]
wavelengths = [int(c.split('_')[1]) for c in roi_cols]
import numpy as np
roi_broad = [c for c, wl in zip(roi_cols, wavelengths) if 320 <= wl <= 480]
roi_narrow = [c for c, wl in zip(roi_cols, wavelengths) if 360 <= wl <= 410]
trimmed['occupancy_broad'] = 1 - trimmed[roi_broad].mean(axis=1)
trimmed['occupancy_narrow'] = 1 - trimmed[roi_narrow].mean(axis=1)
trimmed[['dose_id', 'angle', 'occupancy_broad', 'occupancy_narrow']].head()

Aggregate occupancy by dose and compare against the chromatogram/DAD concentrations.

In [ ]:
occupancy_dose = trimmed.groupby('dose_id')[['occupancy_broad', 'occupancy_narrow']].mean().reset_index()
dose_merge = dose_summary.merge(occupancy_dose, on='dose_id')
dose_merge[['dose_id', 'occupancy_broad', 'occupancy_narrow', 'dad_total_mg_per_gDW_trimmed_mean']]


Plot dose-level scytonemin concentration against the derived occupancy metrics to illustrate the inversion discussed in the thesis.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(dose_merge['occupancy_broad'], dose_merge['dad_total_mg_per_gDW_trimmed_mean'], marker='o', label='Broad ROI')
ax.plot(dose_merge['occupancy_narrow'], dose_merge['dad_total_mg_per_gDW_trimmed_mean'], marker='s', label='Narrow ROI')
ax.set_xlabel('Reflectance occupancy (1 - mean reflectance)')
ax.set_ylabel('DAD total (mg/g DW)')
ax.set_title('Dose vs reflectance occupancy (sample subset)')
ax.legend()
plt.show()

Plot the precision-weighted concentrations to verify the relationship between chromatogram and DAD estimates.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(precision['chrom_total_mg_per_gDW_trimmed_mean'], precision['dad_total_mg_per_gDW_trimmed_mean'], s=50, alpha=0.8)
ax.set_xlabel('Chromatogram total (mg/g DW)')
ax.set_ylabel('DAD total (mg/g DW)')
ax.set_title('Precision-weighted totals (sample subset)')
plt.show()


Dose-level summary statistics help auditors confirm the trend reported in the thesis.

In [ ]:
dose_summary[['dose_id', 'uva_mw_cm2', 'uvb_mw_cm2', 'dad_total_mg_per_gDW_trimmed_mean', 'chrom_total_mg_per_gDW_trimmed_mean']]


### Next Steps

- Regenerate the full canonical dataset via `make reproduce` (outputs land in `data/reference/reflectance/canonical_dataset/`).
- Cross-reference figures in `scaffold/reflectance/figures/` to ensure the sample pipeline mirrors the published plots.
- Notebook authors can import helpers from `src/reflectance/` for more advanced diagnostics.